In [1]:
#22대 총선 개표결과 속성에 ALIAS 추가 
#속성정보가 추가된 22대총선 shp 파일 생산
import pandas as pd
import os
import shutil
pd.set_option('display.max_rows', 10000)
import numpy as np
import geopandas as gpd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
#22대총선 최종 shp 파일 불러오기
file = gpd.read_file('./shp/선거지도_22대총선_선거구_경계/선거지도_최신선거_선거구')
# file_1 = gpd.read_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/지온파스팀 전달/선거지도_최신선거_선거구(동)')
gdf = gpd.GeoDataFrame(file, crs = file.crs)
# gdf_1 = gpd.GeoDataFrame(file_1, crs = file_1.crs)

In [3]:
#22대총선 개표 속성정보 가져오기
att = pd.read_csv('./info_22lesielection_pastelection.csv')
att.head()

,SIDO_DISTR,SIDO__DISTR,SIDO_NM,VOTE_POP,VOTE_CNT,VOTE_SPLT_POP,VOTE_SPLT_CNT,SCS_CD,SCS_PRT,CAN1,CAN1_PRT,CAN1_CNT,CAN2,CAN2_PRT,CAN2_CNT,CAN3,CAN3_PRT,CAN3_CNT,CAN4,CAN4_PRT,CAN4_CNT
0,강원특별자치도강릉시,강원특별자치도_강릉시,강원특별자치도,183878,120669,63209,1313,권성동,국민의힘,김중남,더불어민주당,51731,권성동,국민의힘,64743,이영랑,개혁신당,2882,0,0,0
1,강원특별자치도동해시태백시삼척시정선군,강원특별자치도_동해시태백시삼척시정선군,강원특별자치도,196849,129564,67285,1633,이철규,국민의힘,한호연,더불어민주당,46674,이철규,국민의힘,78325,류성호,개혁신당,2932,0,0,0
2,강원특별자치도속초시인제군고성군양양군,강원특별자치도_속초시인제군고성군양양군,강원특별자치도,148964,99218,49746,1204,이양수,국민의힘,김도균,더불어민주당,43276,이양수,국민의힘,54738,0,0,0,0,0,0
3,강원특별자치도원주시갑,강원특별자치도_원주시갑,강원특별자치도,159375,103607,55768,1071,박정하,국민의힘,원창묵,더불어민주당,50534,박정하,국민의힘,52002,0,0,0,0,0,0
4,강원특별자치도원주시을,강원특별자치도_원주시을,강원특별자치도,148344,98786,49558,947,송기헌,더불어민주당,송기헌,더불어민주당,52920,김완섭,국민의힘,44919,0,0,0,0,0,0


In [4]:
#데이터 길이 확인 및 키 정보 확인
print(f'att : {len(att)} gdf : {len(gdf)}')
print(att.columns)
print(gdf.columns)

att : 254 gdf : 254
Index(['SIDO_DISTR', 'SIDO__DISTR', 'SIDO_NM', 'VOTE_POP', 'VOTE_CNT',
       'VOTE_SPLT_POP', 'VOTE_SPLT_CNT', 'SCS_CD', 'SCS_PRT', 'CAN1',
       'CAN1_PRT', 'CAN1_CNT', 'CAN2', 'CAN2_PRT', 'CAN2_CNT', 'CAN3',
       'CAN3_PRT', 'CAN3_CNT', 'CAN4', 'CAN4_PRT', 'CAN4_CNT'],
      dtype='object')
Index(['SIDO_NM', 'SIDO_CD', 'DISTRICT', 'ALIAS', 'SIDOCD_int', 'geometry'], dtype='object')


In [5]:
#att, gdf의 선거구명 정보가 같은지 확인 gdf의 SIDO_NM, DISTRICT를 합쳐 SIDO_DISTR 컬럼 구성 후 오름차순정렬, att의 SIDO_DISTR과 비교
#같은 행의 갯수 : 254개
gdf['SIDO_DISTR'] = gdf['SIDO_NM'] + gdf['DISTRICT']
gdf = gdf.sort_values('SIDO_DISTR', ascending=False).reset_index(drop = True)
att = att.sort_values('SIDO_DISTR', ascending=False).reset_index(drop = True)
n = 0
for g, a in zip(gdf['SIDO_DISTR'], att['SIDO_DISTR']):
    n += (g == a)
n

254

#### 데이터 모양 비교 att vs shp : key 대응 완료 


In [6]:
gdf.head()

,SIDO_NM,SIDO_CD,DISTRICT,ALIAS,SIDOCD_int,geometry,SIDO_DISTR
0,충청북도,33,충주시,충주,33,"POLYGON ((292464.769 473704.246, 292448.052 47...",충청북도충주시
1,충청북도,33,청주시흥덕구,청주흥덕,33,"POLYGON ((234928.848 443068.116, 234920.756 44...",충청북도청주시흥덕구
2,충청북도,33,청주시청원구,청주청원,33,"POLYGON ((245159.546 450299.152, 245121.916 45...",충청북도청주시청원구
3,충청북도,33,청주시서원구,청주서원,33,"POLYGON ((240470.786 438229.210, 240461.722 43...",충청북도청주시서원구
4,충청북도,33,청주시상당구,청주상당,33,"POLYGON ((250602.834 437061.347, 250629.055 43...",충청북도청주시상당구


In [7]:
att.head()

,SIDO_DISTR,SIDO__DISTR,SIDO_NM,VOTE_POP,VOTE_CNT,VOTE_SPLT_POP,VOTE_SPLT_CNT,SCS_CD,SCS_PRT,CAN1,CAN1_PRT,CAN1_CNT,CAN2,CAN2_PRT,CAN2_CNT,CAN3,CAN3_PRT,CAN3_CNT,CAN4,CAN4_PRT,CAN4_CNT
0,충청북도충주시,충청북도_충주시,충청북도,181005,119216,61789,1220,이종배,국민의힘,김경욱,더불어민주당,57682,이종배,국민의힘,60314,0,0,0,0,0,0
1,충청북도청주시흥덕구,충청북도_청주시흥덕구,충청북도,231288,141730,89558,1925,이연희,더불어민주당,이연희,더불어민주당,72375,김동원,국민의힘,62334,김기영,개혁신당,5096,0,0,0
2,충청북도청주시청원구,충청북도_청주시청원구,충청북도,158647,100091,58556,1342,송재봉,더불어민주당,송재봉,더불어민주당,52620,김수민,국민의힘,46129,0,0,0,0,0,0
3,충청북도청주시서원구,충청북도_청주시서원구,충청북도,161145,105939,55206,1416,이광희,더불어민주당,이광희,더불어민주당,54835,김진모,국민의힘,49688,0,0,0,0,0,0
4,충청북도청주시상당구,충청북도_청주시상당구,충청북도,169036,109208,59828,1143,이강일,더불어민주당,이강일,더불어민주당,55602,서승우,국민의힘,49905,송상호,녹색정의당,1761,우근헌,무소속,797


In [8]:
pd.concat([gdf['SIDO_DISTR'], att['SIDO_DISTR']], axis = 1).head(20)

,SIDO_DISTR,SIDO_DISTR
0,충청북도충주시,충청북도충주시
1,충청북도청주시흥덕구,충청북도청주시흥덕구
2,충청북도청주시청원구,충청북도청주시청원구
3,충청북도청주시서원구,충청북도청주시서원구
4,충청북도청주시상당구,충청북도청주시상당구
5,충청북도증평군진천군음성군,충청북도증평군진천군음성군
6,충청북도제천시단양군,충청북도제천시단양군
7,충청북도보은군옥천군영동군괴산군,충청북도보은군옥천군영동군괴산군
8,충청남도홍성군예산군,충청남도홍성군예산군
9,충청남도천안시을,충청남도천안시을


#### shp, att merge 

In [9]:
#att에서 shp에 중복되는 컬럼을 제외해야함
# Index(['SIDO_DISTR', 'SIDO__DISTR', 'VOTE_POP', 'VOTE_CNT',
#        'VOTE_SPLT_POP', 'VOTE_SPLT_CNT', 'SCS_CD', 'SCS_PRT', 'CAN1',
#        'CAN1_PRT', 'CAN1_CNT', 'CAN2', 'CAN2_PRT', 'CAN2_CNT', 'CAN3',
#        'CAN3_PRT', 'CAN3_CNT', 'CAN4', 'CAN4_PRT', 'CAN4_CNT'],
#       dtype='object')
# Index(['SIDO_NM', 'SIDO_CD', 'DISTRICT', 'ALIAS', 'SIDOCD_int', 'geometry'], dtype='object')

In [10]:
print(f'att : {len(att)} gdf : {len(gdf)}')
gdf_att = gdf.merge(att[['SIDO_DISTR', 'SIDO__DISTR', 'VOTE_POP', 'VOTE_CNT',
                        'VOTE_SPLT_POP', 'VOTE_SPLT_CNT', 'SCS_CD', 'SCS_PRT', 'CAN1',
                        'CAN1_PRT', 'CAN1_CNT', 'CAN2', 'CAN2_PRT', 'CAN2_CNT', 'CAN3',
                        'CAN3_PRT', 'CAN3_CNT', 'CAN4', 'CAN4_PRT', 'CAN4_CNT']],
                          how = 'left', left_on = 'SIDO_DISTR', right_on = 'SIDO_DISTR')
print(len(gdf_att))

att : 254 gdf : 254
254


In [11]:
gdf_att.columns

Index(['SIDO_NM', 'SIDO_CD', 'DISTRICT', 'ALIAS', 'SIDOCD_int', 'geometry',
       'SIDO_DISTR', 'SIDO__DISTR', 'VOTE_POP', 'VOTE_CNT', 'VOTE_SPLT_POP',
       'VOTE_SPLT_CNT', 'SCS_CD', 'SCS_PRT', 'CAN1', 'CAN1_PRT', 'CAN1_CNT',
       'CAN2', 'CAN2_PRT', 'CAN2_CNT', 'CAN3', 'CAN3_PRT', 'CAN3_CNT', 'CAN4',
       'CAN4_PRT', 'CAN4_CNT'],
      dtype='object')

In [12]:
#컬럼순서 및 컬럼명 조정
gdf_att = gdf_att[['ALIAS', 'SIDO_CD', 'DISTRICT','SIDO_DISTR', 'SIDO__DISTR', 'VOTE_POP', 'VOTE_CNT', 'VOTE_SPLT_POP',
                    'VOTE_SPLT_CNT', 'SCS_CD', 'SCS_PRT', 
                    'CAN1', 'CAN1_PRT', 'CAN1_CNT',
                    'CAN2', 'CAN2_PRT', 'CAN2_CNT', 
                    'CAN3', 'CAN3_PRT', 'CAN3_CNT',
                    'CAN4','CAN4_PRT', 'CAN4_CNT',
                    'SIDO_NM', 'geometry']]
gdf_att.columns = ['ALIAS', 'SIDO_CD', 'DISTRICT','SIDO_DISTR', 'SIDO__DISTR', 'VOTE_POP', 'VOTE_CNT', 'SPLT_POP',
                    'SPLT_CNT', 'SCS_CD', 'SCS_PRT', 
                    'CAN1', 'CAN1_PRT', 'CAN1_CNT',
                    'CAN2', 'CAN2_PRT', 'CAN2_CNT', 
                    'CAN3', 'CAN3_PRT', 'CAN3_CNT',
                    'CAN4','CAN4_PRT', 'CAN4_CNT',
                    'SIDO_NM', 'geometry']

In [13]:
gdf_att.head()

,ALIAS,SIDO_CD,DISTRICT,SIDO_DISTR,SIDO__DISTR,VOTE_POP,VOTE_CNT,SPLT_POP,SPLT_CNT,SCS_CD,SCS_PRT,CAN1,CAN1_PRT,CAN1_CNT,CAN2,CAN2_PRT,CAN2_CNT,CAN3,CAN3_PRT,CAN3_CNT,CAN4,CAN4_PRT,CAN4_CNT,SIDO_NM,geometry
0,충주,33,충주시,충청북도충주시,충청북도_충주시,181005,119216,61789,1220,이종배,국민의힘,김경욱,더불어민주당,57682,이종배,국민의힘,60314,0,0,0,0,0,0,충청북도,"POLYGON ((292464.769 473704.246, 292448.052 47..."
1,청주흥덕,33,청주시흥덕구,충청북도청주시흥덕구,충청북도_청주시흥덕구,231288,141730,89558,1925,이연희,더불어민주당,이연희,더불어민주당,72375,김동원,국민의힘,62334,김기영,개혁신당,5096,0,0,0,충청북도,"POLYGON ((234928.848 443068.116, 234920.756 44..."
2,청주청원,33,청주시청원구,충청북도청주시청원구,충청북도_청주시청원구,158647,100091,58556,1342,송재봉,더불어민주당,송재봉,더불어민주당,52620,김수민,국민의힘,46129,0,0,0,0,0,0,충청북도,"POLYGON ((245159.546 450299.152, 245121.916 45..."
3,청주서원,33,청주시서원구,충청북도청주시서원구,충청북도_청주시서원구,161145,105939,55206,1416,이광희,더불어민주당,이광희,더불어민주당,54835,김진모,국민의힘,49688,0,0,0,0,0,0,충청북도,"POLYGON ((240470.786 438229.210, 240461.722 43..."
4,청주상당,33,청주시상당구,충청북도청주시상당구,충청북도_청주시상당구,169036,109208,59828,1143,이강일,더불어민주당,이강일,더불어민주당,55602,서승우,국민의힘,49905,송상호,녹색정의당,1761,우근헌,무소속,797,충청북도,"POLYGON ((250602.834 437061.347, 250629.055 43..."


In [14]:
gdf_att.groupby(['SCS_PRT']).count()

,ALIAS,SIDO_CD,DISTRICT,SIDO_DISTR,SIDO__DISTR,VOTE_POP,VOTE_CNT,SPLT_POP,SPLT_CNT,SCS_CD,CAN1,CAN1_PRT,CAN1_CNT,CAN2,CAN2_PRT,CAN2_CNT,CAN3,CAN3_PRT,CAN3_CNT,CAN4,CAN4_PRT,CAN4_CNT,SIDO_NM,geometry
SCS_PRT,,,,,,,,,,,,,,,,,,,,,,,,
개혁신당,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
국민의힘,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90
더불어민주당,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161
새로운미래,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
진보당,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [40]:
gdf_att.to_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/shp/행정경계_202402/22대총선_선거구/22대총선_선거구_속성_무효표', encoding = 'utf-8')

/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_74489/3378272725.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_att.to_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/shp/행정경계_202402/22대총선_선거구/22대총선_선거구_속성_무효표', encoding = 'utf-8')


In [41]:
len(gdf_att)

254

In [42]:
#22대 유권자_연령별 데이터 불러오기
df_pop = pd.read_csv('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/유권자/22대총선/2024년03월_총선_인구.csv')
len(df_pop)

254

In [48]:
#전국평균 컬럼추가
df_pop.head(2)
li_col = ['age_20','age_30','age_40','age_50','age_60','age_70']
for i in li_col:
    print(i)
    df_pop[f'mn_{i}'] = df_pop[i].mean()
df_pop.head(2)

age_20
age_30
age_40
age_50
age_60
age_70


,SIDO_DISTR,ALIAS,age_20,age_30,age_40,age_50,age_60,age_70,age_sum,mn_age_20,mn_age_30,mn_age_40,mn_age_50,mn_age_60,mn_age_70
0,충청북도_충주시,충주,22284.0,28822.0,36613.0,36693.0,31821.0,24908.0,181141.0,25796.700787,30966.299213,34332.57874,30313.972441,25224.090551,27655.748031
1,충청북도_청주시흥덕구,청주흥덕,44080.0,44155.0,44597.0,33712.0,22124.0,42771.0,231439.0,25796.700787,30966.299213,34332.57874,30313.972441,25224.090551,27655.748031


In [53]:
gdf_att_pop = gdf_att.merge(df_pop.iloc[:,1:], how = 'left', left_on='ALIAS', right_on = 'ALIAS')
len(gdf_att_pop)

254

In [54]:
gdf_att_pop.head(2)

,ALIAS,SIDO_CD,DISTRICT,SIDO_DISTR,SIDO__DISTR,VOTE_POP,VOTE_CNT,SPLT_POP,SPLT_CNT,SCS_CD,SCS_PRT,CAN1,CAN1_PRT,CAN1_CNT,CAN2,CAN2_PRT,CAN2_CNT,CAN3,CAN3_PRT,CAN3_CNT,CAN4,CAN4_PRT,CAN4_CNT,SIDO_NM,geometry,age_20,age_30,age_40,age_50,age_60,age_70,age_sum,mn_age_20,mn_age_30,mn_age_40,mn_age_50,mn_age_60,mn_age_70
0,충주,33,충주시,충청북도충주시,충청북도_충주시,181005,119216,61789,1220,이종배,국민의힘,김경욱,더불어민주당,57682,이종배,국민의힘,60314,0,0,0,0,0,0,충청북도,"POLYGON ((292464.769 473704.246, 292448.052 47...",22284.0,28822.0,36613.0,36693.0,31821.0,24908.0,181141.0,25796.700787,30966.299213,34332.57874,30313.972441,25224.090551,27655.748031
1,청주흥덕,33,청주시흥덕구,충청북도청주시흥덕구,충청북도_청주시흥덕구,231288,141730,89558,1925,이연희,더불어민주당,이연희,더불어민주당,72375,김동원,국민의힘,62334,김기영,개혁신당,5096,0,0,0,충청북도,"POLYGON ((234928.848 443068.116, 234920.756 44...",44080.0,44155.0,44597.0,33712.0,22124.0,42771.0,231439.0,25796.700787,30966.299213,34332.57874,30313.972441,25224.090551,27655.748031


In [58]:
gdf_att_pop.drop(['SIDO__DISTR'], axis = 1).to_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/shp/행정경계_202402/22대총선_선거구/22대총선_선거구_속성_무효표_연령', encoding = 'utf-8')
gdf_att_pop.drop(['SPLT_CNT','SPLT_POP'], axis = 1).to_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/shp/행정경계_202402/22대총선_선거구/선거지도_22대총선_선거구', encoding = 'utf-8')


/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_74489/2156372764.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_att_pop.drop(['SPLT_CNT','SPLT_POP'], axis = 1).to_file('/Users/kownkihoon/Desktop/업무_wavus/선거구지도작성/shp/행정경계_202402/22대총선_선거구/선거지도_22대총선_선거구', encoding = 'utf-8')
